In [2]:
# Cell 1: Setup, Engine, and First Interaction

from sqlmodel import create_engine, Session, SQLModel, select
from sqlalchemy import event
from sqlalchemy.engine import Engine

# --- Import models from your script ---
# (Assumes 'agile_sqlmodel.py' is in the same directory)
from agile_sqlmodel import (
    Product,
    Team,
    TeamMember
    # Import other models like UserStory, Task, etc.,
    # in future cells as you need them.
)

# --- Re-define the Pragma function ---
# (This is needed for SQLite to enforce foreign keys)
@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, _connection_record):
    """Enforce foreign key constraints on SQLite."""
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

# --- Create the Engine (without 'echo=True' for a clean notebook) ---
DB_URL = "sqlite:///./agile_simple.db"
engine = create_engine(DB_URL, echo=False)

# --- A simple function to create tables ---
def create_tables():
    """Create all tables in the database if they don't exist."""
    # This finds all classes that inherited from SQLModel
    SQLModel.metadata.create_all(engine)
    print("Tables checked/created.")

# --- Run the setup ---
create_tables()

print("\n--- Running first transaction ---")

# --- Create your first objects ---
# We use a 'with' block for a clean, safe transaction
with Session(engine) as session:
    
    # Check if data already exists to avoid duplicates
    existing_prod = session.exec(
        select(Product).where(Product.name == "Project Phoenix")
    ).first()

    if not existing_prod:
        print("Creating new root data (Product, Team, Member)...")
        
        # 1. Create root objects that don't need other objects
        prod_1 = Product(name="Project Phoenix")
        team_1 = Team(name="Core Team")
        member_1 = TeamMember(name="Alice", email="alice@phoenix.com")

        # 2. Add them to the session
        session.add(prod_1)
        session.add(team_1)
        session.add(member_1)
        
        # 3. Commit the transaction to the database
        session.commit()
        print("Commit successful.")
        
        # 4. Refresh to get IDs generated by the database
        session.refresh(prod_1)
        session.refresh(team_1)
        session.refresh(member_1)
        
        print(f"\nCreated Product ID: {prod_1.product_id}")
        print(f"Created Team ID: {team_1.team_id}")
        print(f"Created Member ID: {member_1.member_id}")
    else:
        print("Data already exists. Skipping creation.")
        print(f"Found Product ID: {existing_prod.product_id}")

print("\n--- First cell complete. You are ready to query. ---")

Tables checked/created.

--- Running first transaction ---
Creating new root data (Product, Team, Member)...
Commit successful.

Created Product ID: 1
Created Team ID: 1
Created Member ID: 1

--- First cell complete. You are ready to query. ---


In [3]:
# Cell 2: Creating Many-to-Many Relationships

from sqlmodel import Session, select
from agile_sqlmodel import (
    engine,  # Import the engine from Cell 1
    Product,
    Team,
    TeamMember,
    TeamMembership,  # Import the link models
    ProductTeam
)

print("--- Running transaction to link objects ---")

# We'll use the IDs we got from Cell 1
PRODUCT_ID = 1
TEAM_ID = 1
MEMBER_ID = 1

with Session(engine) as session:
    
    # 1. Get the objects from the database
    product = session.get(Product, PRODUCT_ID)
    team = session.get(Team, TEAM_ID)
    member = session.get(TeamMember, MEMBER_ID)

    if not (product and team and member):
        print("Error: Could not find root data. Please re-run Cell 1.")
    else:
        # 2. Create the relationships by appending objects
        
        # Link Member 1 to Team 1
        # (if not already linked)
        if member not in team.members:
            team.members.append(member)
            print(f"Linking Member '{member.name}' to Team '{team.name}'...")
        else:
            print(f"Member '{member.name}' is already linked to Team '{team.name}'.")

        # Link Team 1 to Product 1
        # (if not already linked)
        if team not in product.teams:
            product.teams.append(team)
            print(f"Linking Team '{team.name}' to Product '{product.name}'...")
        else:
            print(f"Team '{team.name}' is already linked to Product '{product.name}'.")

        # 3. Add the "changed" objects and commit
        # SQLModel is smart enough to see the relationship changes
        session.add(product)
        session.add(team)
        session.commit()

        print("Relationships committed.")

# --- Verification Step ---
# Let's query the objects back to prove the links were made

print("\n--- Verifying Relationships ---")
with Session(engine) as session:
    
    # Get the team and its members
    team_with_members = session.get(Team, TEAM_ID)
    print(f"Querying Team: '{team_with_members.name}'")
    for m in team_with_members.members:
        print(f"  -> Has Member: {m.name} (Email: {m.email})")

    # Get the product and its teams
    product_with_teams = session.get(Product, PRODUCT_ID)
    print(f"\nQuerying Product: '{product_with_teams.name}'")
    for t in product_with_teams.teams:
        print(f"  -> Has Team: {t.name}")

print("\n--- Cell 2 complete. ---")

--- Running transaction to link objects ---
2025-11-09 11:12:24,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-09 11:12:24,314 INFO sqlalchemy.engine.Engine SELECT products.product_id AS products_product_id, products.name AS products_name, products.description AS products_description, products.vision AS products_vision, products.roadmap AS products_roadmap, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.product_id = ?
2025-11-09 11:12:24,314 INFO sqlalchemy.engine.Engine [generated in 0.00040s] (1,)
2025-11-09 11:12:24,316 INFO sqlalchemy.engine.Engine SELECT teams.team_id AS teams_team_id, teams.name AS teams_name, teams.created_at AS teams_created_at, teams.updated_at AS teams_updated_at 
FROM teams 
WHERE teams.team_id = ?
2025-11-09 11:12:24,316 INFO sqlalchemy.engine.Engine [generated in 0.00033s] (1,)
2025-11-09 11:12:24,317 INFO sqlalchemy.engine.Engine SELECT team_members.member_id AS team_member

In [4]:
# Cell 3: Building the Work Hierarchy (Theme, Epic, Feature)

from sqlmodel import Session, select
from agile_sqlmodel import (
    engine,
    Product,
    Theme,
    Epic,
    Feature
)

print("--- Running transaction to build hierarchy ---")

PRODUCT_ID = 1
FEATURE_TITLE = "User Login System"

with Session(engine) as session:
    
    # 1. Get the parent Product
    product = session.get(Product, PRODUCT_ID)
    if not product:
        print("Error: Product not found. Please re-run Cell 1.")
    else:
        # 2. Check if the hierarchy already exists
        existing_feature = session.exec(
            select(Feature).where(Feature.title == FEATURE_TITLE)
        ).first()

        if not existing_feature:
            print(f"Creating new hierarchy for Product: '{product.name}'")
            
            # 3. Create the Theme
            # A Theme belongs to a Product
            theme = Theme(
                title="Q1: User Growth",
                description="Initiatives to grow the user base in Q1.",
                product_id=product.product_id  # Link to the product
            )
            session.add(theme)
            session.commit()
            session.refresh(theme)
            print(f"  -> Created Theme (ID: {theme.theme_id}): {theme.title}")

            # 4. Create the Epic
            # An Epic belongs to a Theme
            epic = Epic(
                title="Identity & Access Management",
                summary="Build all login, registration, and auth features.",
                theme_id=theme.theme_id  # Link to the theme
            )
            session.add(epic)
            session.commit()
            session.refresh(epic)
            print(f"  -> Created Epic (ID: {epic.epic_id}): {epic.title}")

            # 5. Create the Feature
            # A Feature belongs to an Epic
            feature = Feature(
                title=FEATURE_TITLE,
                description="Allow users to log in with email and password.",
                epic_id=epic.epic_id  # Link to the epic
            )
            session.add(feature)
            session.commit()
            session.refresh(feature)
            print(f"  -> Created Feature (ID: {feature.feature_id}): {feature.title}")

            print("\nHierarchy created successfully.")

        else:
            print("Hierarchy already exists. Skipping creation.")

# --- Verification Step ---
print("\n--- Verifying Hierarchy ---")
with Session(engine) as session:
    feature = session.exec(
        select(Feature).where(Feature.title == FEATURE_TITLE)
    ).one()
    
    # SQLModel's 'Relationship' feature lets you
    # walk up the chain automatically!
    print(f"Feature: '{feature.title}' (ID: {feature.feature_id})")
    print(f"  -> Belongs to Epic:   '{feature.epic.title}' (ID: {feature.epic_id})")
    print(f"  -> Belongs to Theme:  '{feature.epic.theme.title}' (ID: {feature.epic.theme_id})")
    print(f"  -> Belongs to Product: '{feature.epic.theme.product.name}' (ID: {feature.epic.theme.product_id})")

print("\n--- Cell 3 complete. ---")

--- Running transaction to build hierarchy ---
2025-11-09 11:14:26,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-09 11:14:26,044 INFO sqlalchemy.engine.Engine SELECT products.product_id AS products_product_id, products.name AS products_name, products.description AS products_description, products.vision AS products_vision, products.roadmap AS products_roadmap, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.product_id = ?
2025-11-09 11:14:26,044 INFO sqlalchemy.engine.Engine [cached since 121.7s ago] (1,)
2025-11-09 11:14:26,045 INFO sqlalchemy.engine.Engine SELECT features.feature_id, features.title, features.description, features.created_at, features.updated_at, features.epic_id 
FROM features 
WHERE features.title = ?
2025-11-09 11:14:26,045 INFO sqlalchemy.engine.Engine [generated in 0.00021s] ('User Login System',)
Creating new hierarchy for Product: 'Project Phoenix'
2025-11-09 11:14:26,046 INFO sql

In [6]:
# Cell 4: Visualizing All Populated Tables

import pandas as pd
from IPython.display import display, HTML
from agile_sqlmodel import engine

# List of tables we've added data to so far
table_names = [
    "products",
    "teams",
    "team_members",
    "team_memberships",
    "product_teams",
    "themes",
    "epics",
    "features"
]

print("--- Querying all populated tables ---")

with engine.connect() as connection:
    for table_name in table_names:
        
        # Use pandas to read the SQL query directly
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, connection)
        
        # Display the table name and the DataFrame
        display(HTML(f"<h3>Table: {table_name}</h3>"))
        if df.empty:
            print("(This table is empty)")
        else:
            display(df)

print("\n--- Cell 4 complete. All current data is displayed above. ---")

--- Querying all populated tables ---
2025-11-09 11:15:04,925 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-09 11:15:04,926 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM products")
2025-11-09 11:15:04,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,927 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM products")
2025-11-09 11:15:04,928 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,928 INFO sqlalchemy.engine.Engine SELECT * FROM products
2025-11-09 11:15:04,929 INFO sqlalchemy.engine.Engine [raw sql] ()


,product_id,name,description,vision,roadmap,created_at,updated_at
0,1,Project Phoenix,None,None,None,2025-11-09 10:11:21.172481,2025-11-09 10:11:21.172490


2025-11-09 11:15:04,942 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM teams")
2025-11-09 11:15:04,943 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,943 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM teams")
2025-11-09 11:15:04,943 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,944 INFO sqlalchemy.engine.Engine SELECT * FROM teams
2025-11-09 11:15:04,944 INFO sqlalchemy.engine.Engine [raw sql] ()


,team_id,name,created_at,updated_at
0,1,Core Team,2025-11-09 10:11:21.172554,2025-11-09 10:11:21.172558


2025-11-09 11:15:04,947 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM team_members")
2025-11-09 11:15:04,947 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,948 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM team_members")
2025-11-09 11:15:04,948 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,948 INFO sqlalchemy.engine.Engine SELECT * FROM team_members
2025-11-09 11:15:04,948 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2025-11-09 11:15:04,963 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM epics")
2025-11-09 11:15:04,963 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,964 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM epics")
2025-11-09 11:15:04,964 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,964 INFO sqlalchemy.engine.Engine SELECT * FROM epics
2025-11-09 11:15:04,964 INFO sqlalchemy.engine.Engine [raw sql] ()


,epic_id,title,summary,created_at,updated_at,theme_id
0,1,Identity & Access Management,"Build all login, registration, and auth features.",2025-11-09 10:14:26.050416,2025-11-09 10:14:26.050424,1


2025-11-09 11:15:04,967 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM features")
2025-11-09 11:15:04,967 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,968 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM features")
2025-11-09 11:15:04,968 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-09 11:15:04,968 INFO sqlalchemy.engine.Engine SELECT * FROM features
2025-11-09 11:15:04,968 INFO sqlalchemy.engine.Engine [raw sql] ()


,feature_id,title,description,created_at,updated_at,epic_id
0,1,User Login System,Allow users to log in with email and password.,2025-11-09 10:14:26.054668,2025-11-09 10:14:26.054674,1


2025-11-09 11:15:04,971 INFO sqlalchemy.engine.Engine ROLLBACK

--- Cell 4 complete. All current data is displayed above. ---


In [7]:
# Cell 5: Adding Vision and Roadmap to a Product

import pandas as pd
from IPython.display import display, HTML
from sqlmodel import Session, select
from agile_sqlmodel import engine, Product

PRODUCT_ID = 1

print(f"--- Updating Product ID: {PRODUCT_ID} ---")

with Session(engine) as session:
    
    # 1. Get the product
    product = session.get(Product, PRODUCT_ID)

    if product:
        # 2. Update the fields
        print("Setting vision and roadmap...")
        product.vision = "To be the leading platform for remote team collaboration."
        product.roadmap = "Q1: User Login. Q2: Team Dashboards. Q3: Real-time Chat."
        
        # 3. Add the object (to mark it as 'dirty') and commit
        session.add(product)
        session.commit()
        session.refresh(product)
        
        print("Product updated successfully.")
    else:
        print(f"Error: Product {PRODUCT_ID} not found.")

# --- Verification Step ---
print("\n--- Verifying Update (Reading from 'products' table) ---")

with engine.connect() as connection:
    query = f"SELECT * FROM products WHERE product_id = {PRODUCT_ID}"
    df = pd.read_sql(query, connection)
    
    # Display the DataFrame
    display(HTML(f"<h3>Table: products (ID: {PRODUCT_ID})</h3>"))
    display(df)

print("\n--- Cell 5 complete. ---")

--- Updating Product ID: 1 ---
2025-11-09 11:17:30,289 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-09 11:17:30,290 INFO sqlalchemy.engine.Engine SELECT products.product_id AS products_product_id, products.name AS products_name, products.description AS products_description, products.vision AS products_vision, products.roadmap AS products_roadmap, products.created_at AS products_created_at, products.updated_at AS products_updated_at 
FROM products 
WHERE products.product_id = ?
2025-11-09 11:17:30,290 INFO sqlalchemy.engine.Engine [cached since 306s ago] (1,)
Setting vision and roadmap...
2025-11-09 11:17:30,292 INFO sqlalchemy.engine.Engine UPDATE products SET vision=?, roadmap=?, updated_at=CURRENT_TIMESTAMP WHERE products.product_id = ?
2025-11-09 11:17:30,292 INFO sqlalchemy.engine.Engine [generated in 0.00040s] ('To be the leading platform for remote team collaboration.', 'Q1: User Login. Q2: Team Dashboards. Q3: Real-time Chat.', 1)
2025-11-09 11:17:30,293 INFO sqlalchem

,product_id,name,description,vision,roadmap,created_at,updated_at
0,1,Project Phoenix,None,To be the leading platform for remote team col...,Q1: User Login. Q2: Team Dashboards. Q3: Real-...,2025-11-09 10:11:21.172481,2025-11-09 10:17:30


2025-11-09 11:17:30,300 INFO sqlalchemy.engine.Engine ROLLBACK

--- Cell 5 complete. ---


In [1]:
from tools.orchestrator_tools import count_projects, list_projects, get_project_details

projects_summary = count_projects()
projects_list = list_projects()


# Get details for a specific project
project_details = get_project_details(product_id=1)
print("Project Details:", project_details)

2025-11-11 16:57:01,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-11 16:57:02,003 INFO sqlalchemy.engine.Engine SELECT products.product_id, products.name, products.description, products.vision, products.roadmap, products.created_at, products.updated_at 
FROM products
2025-11-11 16:57:02,004 INFO sqlalchemy.engine.Engine [generated in 0.00039s] ()
2025-11-11 16:57:02,004 INFO sqlalchemy.engine.Engine ROLLBACK
2025-11-11 16:57:02,005 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-11 16:57:02,005 INFO sqlalchemy.engine.Engine SELECT products.product_id, products.name, products.description, products.vision, products.roadmap, products.created_at, products.updated_at 
FROM products
2025-11-11 16:57:02,005 INFO sqlalchemy.engine.Engine [cached since 0.001979s ago] ()
2025-11-11 16:57:02,006 INFO sqlalchemy.engine.Engine SELECT user_stories.story_id, user_stories.title, user_stories.story_description, user_stories.acceptance_criteria, user_stories.status, user_stories.stor

In [2]:
print("Projects Summary:", projects_summary.get('count'))
print("Projects List:", projects_list.get('projects'))


Projects Summary: 1
Projects List: [{'product_id': 1, 'name': 'Project Phoenix', 'vision': 'To be the leading platform for remote team collaboration.', 'roadmap': 'Q1: User Login. Q2: Team Dashboards. Q3: Real-time Chat.', 'user_stories_count': 0}]


In [10]:
# Cell 1: Wire the persistent session service into the runner
from google.adk.runners import Runner

APP_NAME = "orchestrator_app"
USER_ID = "alex"
SESSION_ID = "orchestrator-session-1"

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,  # <-- your DatabaseSessionService("sqlite:///agile_sqlmodel.db")
)

# continue an existing session if present; otherwise create it with initial state
existing = session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)
if existing and existing.sessions:
    SESSION_ID = existing.sessions[0].id
else:
    session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state=initial_state,  # your computed initial state
    )


NameError: name 'root_agent' is not defined